# Problema 4: Producción del Sector TI

El sector de TI de una empresa posee 3 clústeres que son utilizados para determinadas tareas. La empresa desea **maximizar la ganancia** de las tareas (los valores de la ganancia se encuentran en dolares).
  
<img src="capacidad_maquinas.png" width= 700>

¿Cuantas tareas pueden ser ejecutada con el objetivo de no violar la capacidad disponible de las maquinas por mes?

## Variables de decisión

* $x_{1}$: Número de tareas 1
* $x_{2}$: Número de tareas 2

* Función Objetivo = $5*x_{1} + 9*x_{2}$ .

## Instalación de Paquetes

In [45]:
# https://deap.readthedocs.io/en/master/
!pip install deap

In [46]:
# Bibliotecas a serem utilizadas
import random
import numpy as np
from deap import algorithms, base, creator, tools

## Preparación para Optimización

1. **Función objetivo** 
2. **Variables de decisión**
3. **Operadores**

In [47]:
# Función Objetivo - Calculo de la ganancia
def objetive_function(individual):
  #codigo de la función
  #resultado retorna como tupla (result)
  return (individual[0]*5 + individual[1]*9),

In [48]:
#Ref: https://deap.readthedocs.io/en/master/tutorials/advanced/constraints.html?highlight=feasible#penalty-function  
def feasible(individual):
  if individual[0]*10 + individual[1]*15 > 4200*60:
    return False
  if individual[1]*12 > 500*60:
    return False
  if individual[0]*4 + individual[1]*6 > 1800*60:
    return False
  return True 

In [49]:
def distance(individual):
  constraint1 = abs(individual[0]*10 + individual[1]*15 - 4200*60)
  constraint2 = abs(individual[1]*12 - 500*60)
  constraint3 = abs(individual[0]*4 + individual[1]*6 - 1800*60)
  return constraint1 + constraint2 + constraint3   

In [50]:
# Creación estructura de fitness e individuo
##para problema de maximización el peso es positivo y minimización el peso es negativo
creator.create("FitnessMax", base.Fitness, weights=(1.0,)) 
creator.create("Individual",list, fitness=creator.FitnessMax)

In [51]:
toolbox = base.Toolbox()

# Generador de atributos reales: nombre, función que genera cada variable, intervalo (limite superior e inferior)
toolbox.register("attr_int", random.randint,0,30000) #genera los numeros entre 0 y 30000

# Generador de individuo
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, 2)

# Generar la población
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [52]:
#Inicializar Operadores: https://deap.readthedocs.io/en/master/api/tools.html
toolbox.register("evaluate",objetive_function)

# Crear Restricciones
toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0, distance))

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=30000, indpb=0.05)
toolbox.register("select",tools.selTournament, tournsize=3)

In [53]:
pop = toolbox.population(n=100)                # inicialização da pop
hof = tools.HallOfFame(1)                      # melhor indivíduo
stats = tools.Statistics(lambda ind:ind.fitness.values) # estatísticas
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Ciclo evolutivo

In [54]:
pop, log =algorithms.eaSimple(population=pop, 
                              toolbox=toolbox, 
                              cxpb=0.5, 
                              mutpb=0.1, 
                              ngen=100, 
                              stats=stats, 
                              halloffame=hof, 
                              verbose=0)

# Resultados

In [55]:
# Mejor Solución
tiempo_ciclo_tarea_1 = 14 # 14 min/ciclo
tiempo_ciclo_tarea_2 = 33 # 33 min/ciclo

print(f"Mejor Individuo: {hof[0]}")

print(f"Número de tareas 1: {hof[0][0]}")
print(f"Número de tareas 2: {hof[0][1]} \n")

#validacióon del mejor individuo
print(f"Ganancia Generada por los tareas: ${objetive_function(hof[0])[0]}")

Mejor Individuo: [21665, 1868]
Número de tareas 1: 21665
Número de tareas 2: 1868 

Ganancia Generada por los tareas: $125137


In [56]:
# Individuo Viable?
feasible(hof[0])

True